## Symbolic solution of ODEs with sympy


In [ ]:
import sympy as sym
import sympy
sym.init_printing() # for LaTeX formatted output

import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#plt.style.use("fivethirtyeight")

import scipy as sp
import matplotlib as mpl       # As of July 2017 Bucknell computers use v. 2.x 

# Following is an Ipython magic command that puts figures in the  notebook.
# For figures in separate windows, comment out following line and uncomment
# the next line
# Must come before defaults are changed.
%matplotlib inline 
#notebook
#%matplotlib
%config InlineBackend.figure_format='retina'

# As of Aug. 2017 reverting to 1.x defaults.
# In 2.x text.ustex requires dvipng, texlive-latex-extra, and texlive-fonts-recommended, 
# which don't seem to be universal
# See https://stackoverflow.com/questions/38906356/error-running-matplotlib-in-latex-type1cm?
mpl.style.use('classic')
        
# M.L. modifications of matplotlib defaults using syntax of v.2.0 
# More info at http://matplotlib.org/2.0.0/users/deflt_style_changes.html
# Changes can also be put in matplotlibrc file, or effected using mpl.rcParams[]
plt.rc('figure', figsize = (6, 4.5))            # Reduces overall size of figures
plt.rc('axes', labelsize=16, titlesize=14)
plt.rc('figure', autolayout = True)             # Adjusts supblot parameters for new size

from helpFunction import *

# Difference Equation

## Motivation

### Saving

Solve  $${ y }_{ 0 },\quad { y }_{ 1 }=b y_0,\quad { y }_{ n }=b{ y }_{ n-1 }$$ 

In [ ]:
x,t,b,c,d, n, r,C0,C1, y0, ybar, N = sym.symbols("x t b c d n r C0 C1 y0 ybar N")
y = sym.Function("y")

### A first order homogeneous difference equation

In [ ]:
ode=sym.Eq(y(n), b*y(n-1))
ode

In [ ]:
sym.rsolve(ode,y(n))

This function is usually referred to as the **complementary function/solution** of the general first-order difference equation, when we consider the modified first-order difference equation in which we simply ignore the right-hand-side term d.


### general first-order difference equation

In [ ]:
ode=sym.Eq(y(n+1) + c*y(n), d)
ode

In [ ]:
ode_sol=sym.rsolve(ode,y(n))
ode_sol

It is called **general solution** as it is independent of a starting/initial value

In [ ]:
ics = {y(0): y0}
ics #initial condition

In [ ]:
C_eq = sym.Eq(y(n).subs(n, 0).subs(ics), ode_sol.subs(n, 0))
C_eq

In [ ]:
C_sol = sym.solve(C_eq)
C_sol

In [ ]:
ode_sol.subs(C_sol[0])

This expression is called the **definite solution** to the general first-order difference equation

### Second Order Linear Difference Equation 2017m2q4

In [ ]:
ode=sym.Eq(y(n+2)- 0.4*y(n+1) -0.1* y(n),N)
ode

In [ ]:
ode_sol=sym.rsolve(ode,y(n))
ode_sol

In [ ]:
sym.solve(r**2-0.4*r-0.1, r)

### Steady States

In [ ]:
ss=sym.Eq(ybar- 0.4*ybar -0.1* ybar,N)
ss

In [ ]:
sym.solve(ss, ybar)

#### Characteristic Equation for Homogeneous Solutions

In [ ]:
ce=sym.Eq(r**2-0.4*r -0.1 ,0)
ce

In [ ]:
rsol=sym.solve(ce)
rsol

#### Dynamics and Steady State

In [ ]:
ode_sol

In [ ]:
ode_sol.subs(n,0)

In [ ]:
ode_sol.subs(n,1)

In [ ]:
constsol = sym.solve([ode_sol.subs(n,0)-20000])
constsol

In [ ]:
ode_sol.subs(constsol)

In [ ]:
ode_sol.subs(constsol).subs(N,10000)

In [ ]:
y_n = sympy.lambdify((n), ode_sol.subs(constsol), 'numpy')
y_n

In [ ]:
t = np.linspace(0, 4, 100)
y_n(t)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

t = np.linspace(0, 10, 100)

for a in [0.02, 0.04, 0.06]:
    ax.scatter(t, y_n(t), label=r"difference equation")

ax.set_title(r"$%s$" % sympy.latex(ode_sol), fontsize=18)
ax.set_xlabel(r"$n$", fontsize=18)
ax.set_ylabel(r"$y$", fontsize=18)
#ax.legend()

fig.tight_layout()

In [ ]:
n, r,C0,C1, y0, ybar = sym.symbols("n, r,C0,C1, y0, ybar")
y = sym.Function("y")
yn = R**n (C0*sym.cos(Rad*n)+ C1*sym.sin(Rad*n)   )
yn

### A first order homogeneous differential equation

In [ ]:
#https://www.youtube.com/watch?v=4X0SGGrXDiI

a, t, y0 = sym.symbols("a t y0")
y = sym.Function("y")

In [ ]:
ode=sym.Eq(y(t).diff(t), a*y(t))
ode

- y: money in bank
- a: interest rate, for example 6%
- t: time year


In [ ]:
sym.classify_ode(ode)

In [ ]:
ode_sol=sym.dsolve(ode, y(t), hint="separable")
ode_sol

In [ ]:
ode_sol.free_symbols

In [ ]:
ics = {y(0): y0}

In [ ]:
def apply_ics(sol, ics, x, known_params):
    """
    Apply the initial conditions (ics), given as a dictionary on
    the form ics = {y(0): y0: y(x).diff(x).subs(x, 0): yp0, ...}
    to the solution of the ODE with indepdendent variable x.
    The undetermined integration constants C1, C2, ... are extracted
    from the free symbols of the ODE solution, excluding symbols in
    the known_params list.
    """
    free_params = sol.free_symbols - set(known_params)
    eqs = [(sol.lhs.diff(x, n) - sol.rhs.diff(x, n)).subs(x, 0).subs(ics)
           for n in range(len(ics))]
    sol_params = sym.solve(eqs, free_params)
    return sol.subs(sol_params)

In [ ]:
apply_ics(ode_sol, ics, t, [a])

### Second Order Differential Equation

#### Define the differential equation as a sym.Eq()

In [ ]:
x,t, b,c,d, n, C0, y0 = sym.symbols("x t  b c d n C0 y0")
f = sym.Function("f")

In [ ]:
diffeq = sym.Eq(f(t).diff(t, t) - 2*f(t).diff(t) + f(t), sym.sin(t))
diffeq

#### Solve differential equation

In [ ]:
soln = sym.dsolve(diffeq,f(t))
soln

#### Boundary conditions
This isn't implemented yet in `dsolve` -- it's on the "to do" list<br>
For now, solve for contants on your own. For example, if
$$ f(0) = 1\quad\mbox{and}\quad\left.\frac{df}{dx}\right|_0 = 0,  $$
solve the following equations:

In [ ]:
constants = sym.solve([soln.rhs.subs(t,0) - 1, soln.rhs.diff(t,1).subs(t,0)- 0])
constants

In [ ]:
C1, C2 = sym.symbols('C1,C2')
soln = soln.subs(constants)
soln

#### Convert soln to python function for numerical evaluation/plotting
I'm not sure why I had to specify the modulue for conversion of sympy functions.<br>
See http://docs.sympy.org/latest/modules/utilities/lambdify.html<br>
In previous examples, sympy figured out a good module "on its own." 

In [ ]:
func = sym.lambdify(t,soln.rhs,'numpy')

In [ ]:
xx = sp.arange(-1,1,.01)  # name = xx so it won't collide with symbol x
y = func(xx)
plt.figure(1)
plt.plot(xx,y);

In [ ]:
from IPython.core.display import HTML, Image
css_file = '../../custom.css'
HTML(open(css_file, 'r').read())

In [ ]:
#%load_ext watermark
%watermark -v -m -p sympy,matplotlib -g